In [1]:
import logging
import tensorflow as tf


GPU_FROM = 2
GPU_TO = 3  

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

2023-01-24 20:01:37.727067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 20:01:37.871185: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 20:01:37.906851: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-24 20:01:38.556734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import string
import html
import tensorflow_datasets as tfds

from segmentation.model import SpaceSegmentationTransformer
from segmentation.model import LossWithVoids

gap = 3
len = 100

punc_mapping = {ord(x):x for x in string.punctuation}
entity_mapping = {f" ?{k}": v for k, v in html.entities.html5.items() if k.endswith(";") and v in string.punctuation}
punc_mapping = {f' ?&?#?{k};': v for k, v in punc_mapping.items()}

train, test = tfds.load('ag_news_subset', split="train"), tfds.load('ag_news_subset', split="test")

def unescape(text):
    for match, replace in punc_mapping.items():
        text = tf.strings.regex_replace(text, match, replace)
    for match, replace in entity_mapping.items():#
        text = tf.strings.regex_replace(text, match, replace)
    return text

def join_title_desc(text_dict):
    return text_dict['title'] + ' ' + text_dict['description']

def remove_space(text):
    return tf.strings.reduce_join(tf.strings.strip(tf.strings.split(text)), axis=-1)

def space_after_letter(letter):
    def replacer(x):
        x = tf.strings.regex_replace(x, f"({letter})", r"\1 ") # add whitespace back in after every letter e
        return x
    return replacer

train = train.map(join_title_desc).map(unescape).map(remove_space)
test = test.map(join_title_desc).map(unescape).map(remove_space)

for x in train.take(1):
    print(x)

/home/ma/a/al3615/tf_210/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-24 20:01:39.865202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 20:01:40.567158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 2, name: GeForce RTX 3090, pci bus id: 0000:86:00.0, compute capability: 8.6


tf.Tensor(b"AMDDebutsDual-CoreOpteronProcessorAMD'snewdual-coreOpteronchipisdesignedmainlyforcorporatecomputingapplications,includingdatabases,Webservices,andfinancialtransactions.", shape=(), dtype=string)


2023-01-24 20:01:41.517033: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:

def generate_labels(text):
    text = tf.strings.substr(text, 0, 100)
    text = tf.strings.lower(text)
    encoder_in = text
    encoder_out = tf.strings.regex_replace(text, r"[^e]", "1") # 1 indicates character
    encoder_out = tf.strings.regex_replace(encoder_out, r"e", "2") # 2 indicates space
    encoder_out = tf.strings.bytes_split(encoder_out)
    encoder_out = tf.strings.to_number(encoder_out)
    encoder_out = tf.pad(encoder_out, [[0,100-tf.shape(encoder_out)[0]]])

    return (encoder_in, None), encoder_out

train = train.map(generate_labels)
val = test.map(generate_labels)
for x in train.take(1):
    print(x)

((<tf.Tensor: shape=(), dtype=string, numpy=b"amddebutsdual-coreopteronprocessoramd'snewdual-coreopteronchipisdesignedmainlyforcorporatecomputinga">, None), <tf.Tensor: shape=(100,), dtype=float32, numpy=
array([1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)>)


2023-01-24 20:01:42.005869: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=100,
    standardize="lower",
    split="character",
    output_mode="int",
)

tokenizer.adapt(train.take(1000).map(lambda x,y: x[0]))
tokenizer.get_vocabulary()

2023-01-24 20:01:45.144471: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


['',
 '[UNK]',
 'e',
 'a',
 't',
 's',
 'o',
 'i',
 'n',
 'r',
 'l',
 'd',
 'h',
 'c',
 'u',
 'p',
 'm',
 'f',
 'g',
 'b',
 'y',
 'w',
 'k',
 'v',
 '-',
 ',',
 '.',
 "'",
 '(',
 ')',
 '0',
 'x',
 'j',
 '2',
 '1',
 'q',
 '&',
 ':',
 'z',
 '4',
 '3',
 '\\',
 '5',
 '"',
 '<',
 '>',
 '/',
 '7',
 '6',
 '$',
 '8',
 '9',
 ';',
 '?',
 '=',
 '#',
 '!',
 '*']

In [5]:
for x in train.take(1):
    print(x)
    tokens = tokenizer(x[0][0])
    print(tokens)
    space_tokens = tf.where(tokens == 2, tokens, 3) # Just happen to know e tokenizes to 2 - messy af
    print(tf.cast(tf.where(x[1] == 2, x[1], 7), dtype=space_tokens.dtype) == space_tokens) # TODO: investigate what is going on here

((<tf.Tensor: shape=(), dtype=string, numpy=b"amddebutsdual-coreopteronprocessoramd'snewdual-coreopteronchipisdesignedmainlyforcorporatecomputinga">, None), <tf.Tensor: shape=(100,), dtype=float32, numpy=
array([1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)>)
tf.Tensor(
[ 3 16 11 11  2 19 14  4  5 11 14  3 10 24 13  6  9  2  6 15  4  2  9  6
  8 15  9  6 13  2  5  5  6  9  3 16 11 27  5  8  2 21 11 14  3 10 24 13
  6  9  2  6 15  4  2  9  6  8 13 12  7 15  7  5 11  2  5  7 18  8  2 11
 16  3  7  8 10 20 17  6  9 13  6  9 15  6  9  3  4  2 13  6 16 15 14  4
  7  8 18  3], shape=(100,

2023-01-24 20:01:45.498410: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
model = SpaceSegmentationTransformer(
    num_layers=2,
    d_model=512,
    num_attention_heads=3,
    seq_len=100,
    dff=1028,
    input_tokenizer=tokenizer,
    dropout_rate=0.1
)

In [7]:
from segmentation.metrics import SparseAccuracyWithIgnore
from segmentation.metrics import SparsePrecision
from segmentation.metrics import SparseRecall
from segmentation.metrics import SparseF1

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        SparseAccuracyWithIgnore(ignore_token=0.),
        SparsePrecision(class_id=2, name="space_precision"),
        SparseRecall(class_id=2, name="space_recall"),
        SparseF1(class_id=2, name="space_f1"),
        SparsePrecision(class_id=1, name="char_precision"),
        SparseRecall(class_id=1, name="char_recall"),
        SparseF1(class_id=1, name="char_f1"),
    ]
)

In [8]:
m = tf.keras.metrics.Precision(class_id=2)
# m.update_state([1., 2., 3., 4.],[4., 2., 2., 4.])
# m.result().numpy()

In [9]:
train_ds = train.shuffle(100).batch(8)
val_ds = val.batch(8)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_space_precision",
        patience=2,
        mode="max"
    )
]

model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks)

Epoch 1/100
    7/15000 [..............................] - ETA: 5:41 - loss: 1.8805 - sparse_categorical_accuracy: 0.7785 - space_precision: 0.0568 - space_recall: 0.0242 - space_f1: 0.0339 - char_precision: 0.8824 - char_recall: 0.8733 - char_f1: 0.8779  

2023-01-24 20:01:50.999388: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


15000/15000 [==============================] - 346s 23ms/step - loss: 1.7429 - sparse_categorical_accuracy: 0.8918 - space_precision: 0.0568 - space_recall: 1.1573e-05 - space_f1: 2.3140e-05 - char_precision: 0.8909 - char_recall: 0.9999 - char_f1: 0.9423 - val_loss: 1.7402 - val_sparse_categorical_accuracy: 0.8920 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.8909 - val_char_recall: 1.0000 - val_char_f1: 0.9423
Epoch 2/100
15000/15000 [==============================] - 339s 23ms/step - loss: 1.7428 - sparse_categorical_accuracy: 0.8919 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.8909 - char_recall: 1.0000 - char_f1: 0.9423 - val_loss: 1.7402 - val_sparse_categorical_accuracy: 0.8920 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.8909 - val_char_recall: 1.0000 - val_char_f1: 0.9423
Epoch 3/100
15000/15000 [========

In [10]:
model.evaluate(val_ds)

950/950 [==============================] - 13s 13ms/step - loss: 1.7402 - sparse_categorical_accuracy: 0.8920 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.8909 - char_recall: 1.0000 - char_f1: 0.9423


[1.740188479423523,
 0.8920347094535828,
 0.0,
 0.0,
 nan,
 0.89094078540802,
 1.0,
 0.9423254132270813]

In [11]:
preds = model([("dunningkruger", "eeeeeeeeeeeeeeeeeeeee", "thequickbrownfoxjumpedoverthelazydog"),(None,None, None)])
preds, tf.argmax(preds, axis=-1)

(<tf.Tensor: shape=(3, 100, 3), dtype=float32, numpy=
 array([[[1.05425354e-16, 1.00000000e+00, 2.74945489e-14],
         [1.08670727e-16, 1.00000000e+00, 2.77842290e-14],
         [1.05154641e-16, 1.00000000e+00, 2.48668443e-14],
         [1.02765776e-16, 1.00000000e+00, 2.47171329e-14],
         [1.00100706e-16, 1.00000000e+00, 2.49156978e-14],
         [1.01054895e-16, 1.00000000e+00, 2.43982504e-14],
         [1.04804247e-16, 1.00000000e+00, 2.39520978e-14],
         [9.52140935e-17, 1.00000000e+00, 2.66245461e-14],
         [9.57075166e-17, 1.00000000e+00, 2.15318740e-14],
         [9.82125239e-17, 1.00000000e+00, 2.65174117e-14],
         [1.02147078e-16, 1.00000000e+00, 2.36837968e-14],
         [6.09608421e-17, 1.00000000e+00, 7.09642712e-14],
         [9.24537918e-17, 1.00000000e+00, 2.20068646e-14],
         [9.35343279e-17, 1.00000000e+00, 2.49943753e-14],
         [9.35714451e-17, 1.00000000e+00, 2.55101065e-14],
         [9.28832970e-17, 1.00000000e+00, 2.59816616e-14],
  

In [12]:
model.encoder.pos_embedding(tokenizer(["e"]))

<tf.Tensor: shape=(1, 100, 512), dtype=float32, numpy=
array([[[ 0.08132914,  0.08854049, -0.14747757, ..., -0.0485481 ,
          1.4191325 ,  1.0945144 ],
        [ 1.9637349 ,  0.90921146, -0.16049284, ...,  2.1188207 ,
          0.3337893 ,  0.89834803],
        [ 2.0315614 ,  1.02377   , -0.00431025, ...,  2.1188207 ,
          0.3337893 ,  0.89834803],
        ...,
        [ 1.5018716 , -0.5380123 , -0.21734512, ...,  2.1187623 ,
          0.333735  ,  0.8982975 ],
        [ 0.548882  ,  0.37240577, -1.0522068 , ...,  2.118761  ,
          0.33373386,  0.8982964 ],
        [ 0.12305707,  1.0375065 , -1.8147955 , ...,  2.1187599 ,
          0.33373272,  0.8982954 ]]], dtype=float32)>

In [13]:
model.encoder.pos_embedding(tokenizer(["d"]))

<tf.Tensor: shape=(1, 100, 512), dtype=float32, numpy=
array([[[-0.26644796,  0.7871986 , -0.2654642 , ...,  2.1398592 ,
          1.6915152 ,  1.4246267 ],
        [ 1.9637349 ,  0.90921146, -0.16049284, ...,  2.1188207 ,
          0.3337893 ,  0.89834803],
        [ 2.0315614 ,  1.02377   , -0.00431025, ...,  2.1188207 ,
          0.3337893 ,  0.89834803],
        ...,
        [ 1.5018716 , -0.5380123 , -0.21734512, ...,  2.1187623 ,
          0.333735  ,  0.8982975 ],
        [ 0.548882  ,  0.37240577, -1.0522068 , ...,  2.118761  ,
          0.33373386,  0.8982964 ],
        [ 0.12305707,  1.0375065 , -1.8147955 , ...,  2.1187599 ,
          0.33373272,  0.8982954 ]]], dtype=float32)>

In [14]:
import lovely_tensors as lt
lt.monkey_patch()
[print(x) for x in model.trainable_variables]

<tf.Variable 'space_segmentation_transformer/encoder/postional_embedding/embedding/embeddings:0' shape=(58, 512) dtype=float32, numpy=
array([[ 0.04959753,  0.00386059, -0.04253489, ...,  0.04944536,
        -0.02944263, -0.00449242],
       [ 0.04138169, -0.02623766,  0.0217234 , ...,  0.01663146,
        -0.03474427, -0.01255412],
       [ 0.00359427,  0.00391297, -0.00651765, ..., -0.04633972,
         0.01852321,  0.00417699],
       ...,
       [ 0.04425664, -0.00089257,  0.02662169, ...,  0.01721764,
        -0.0331916 , -0.04946451],
       [-0.03879231, -0.01136684,  0.02757949, ..., -0.03115348,
        -0.02648364,  0.04941115],
       [ 0.02729204, -0.02474734,  0.04571542, ...,  0.03006331,
         0.00380225, -0.00652245]], dtype=float32)>
<tf.Variable 'space_segmentation_transformer/encoder/encoder_sublayer_0/multi_head_attention/query/kernel:0' shape=(512, 3, 512) dtype=float32, numpy=
array([[[ 6.4165232e-04,  5.6641418e-03,  2.1687355e-03, ...,
         -4.5308599e-04

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]